In [35]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
import numpy as np

In [36]:

IMG_SIZE = (128, 128)
BATCH_SIZE = 32

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    height_shift_range=0.2,
    width_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_dataset = train_datagen.flow_from_directory(
    'train/', 
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

val_dataset = val_datagen.flow_from_directory(
    'valid/',  
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [37]:
print(train_dataset.class_indices)
print(val_dataset.class_indices)

{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___Northern_Leaf_Blight': 9, 'Corn_(maize)___healthy': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13, 'Grape___healthy': 14, 'Orange___Haunglongbing_(Citrus_greening)': 15, 'Peach___Bacterial_spot': 16, 'Peach___healthy': 17, 'Pepper,_bell___Bacterial_spot': 18, 'Pepper,_bell___healthy': 19, 'Potato___Early_blight': 20, 'Potato___Late_blight': 21, 'Potato___healthy': 22, 'Raspberry___healthy': 23, 'Soybean___healthy': 24, 'Squash___Powdery_mildew': 25, 'Strawberry___Leaf_scorch': 26, 'Strawberry___healthy': 27, 'Tomato___Bacterial_spot': 28, 'Tomato___Early_blight': 29, 'Tomato___Late_blight': 30, 'Tomato

In [39]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x) 
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_dataset.num_classes, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_dataset,
    epochs=3,
    validation_data=val_dataset,
    verbose=1
)

Epoch 1/3
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 411s 187ms/step - accuracy: 0.6889 - loss: 1.0619 - val_accuracy: 0.8246 - val_loss: 0.5676
Epoch 2/3
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 400s 182ms/step - accuracy: 0.8112 - loss: 0.5875 - val_accuracy: 0.8214 - val_loss: 0.5597
Epoch 3/3
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 403s 183ms/step - accuracy: 0.8303 - loss: 0.5227 - val_accuracy: 0.8475 - val_loss: 0.4701


In [ ]:
model.save('model.h5')

In [ ]:

model = load_model('model.h5')

loss, accuracy = model.evaluate(val_dataset)

print(f'Test accuracy: {accuracy:.4f}')

/Users/sammamsohail/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


550/550 ━━━━━━━━━━━━━━━━━━━━ 79s 144ms/step - accuracy: 0.9110 - loss: 0.2664
Test accuracy: 0.9132
